In [173]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [174]:
# Параметры для обработки изображений
img_height = 224
img_width = 224
batch_size = 32

In [175]:
dataset_dir = './dataset'
classes = os.listdir(dataset_dir)

print(classes)

['0', '1', '2', 'Wrench', 'Zebra', 'Zucchini']


In [176]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    './dataset',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    './dataset',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


print(train_generator.class_indices)

Found 1055 images belonging to 6 classes.


Found 1055 images belonging to 6 classes.
{'0': 0, '1': 1, '2': 2, 'Wrench': 3, 'Zebra': 4, 'Zucchini': 5}


In [177]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [178]:
if os.path.exists('newModel.keras'):
    model = tf.keras.models.load_model('newModel.keras')
else:
    model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=5,
        validation_data=validation_generator,
        validation_steps=len(validation_generator)
    )

model.save('newModel.keras')

In [179]:
model_load = tf.keras.models.load_model('newModel.keras')

loss, acc = model.evaluate(validation_generator, verbose=2)
print("Validation accuracy:", acc)

c:\Users\adios\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


33/33 - 13s - 390ms/step - accuracy: 0.8882 - loss: 0.3003
Validation accuracy: 0.8881516456604004


In [180]:
def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_batch)

new_image = load_and_preprocess_image('Salix_alba_Morton.jpg')

In [181]:
preds = model.predict(new_image)
predicted_class = np.argmax(preds)

print("Predicted class:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted class: 2


In [ ]:
def find_similar_images(class_index, n=5):
    similar_images = []
    # Заменить class_index на class_name
    # Ограничиваем поиск пятью изображениями
    for root, dirs, files in os.walk(f'dataset/{class_index}'):
        for file in files[:n]:
            image_path = os.path.join(root, file)
            similar_images.append(image_path)
            print(image_path)

    print(similar_images)        
    return similar_images

In [183]:
similar_images = find_similar_images(predicted_class)

dataset/2\00dc8d7b443a2317.jpg
dataset/2\011d98c29ca02598.jpg
dataset/2\01e18bb533565ca8.jpg
dataset/2\0267ce45e0bbd590.jpg
dataset/2\033a4804f8b1f38c.jpg
['dataset/2\\00dc8d7b443a2317.jpg', 'dataset/2\\011d98c29ca02598.jpg', 'dataset/2\\01e18bb533565ca8.jpg', 'dataset/2\\0267ce45e0bbd590.jpg', 'dataset/2\\033a4804f8b1f38c.jpg']
